# **Progetto di Matteo Calvanico e Filippo Monti**

---
---

## Per quanto riguarda i CSV...

Per questo progetto useremo due CSV, eccoli nel dettaglio:
#### Fonti

> * [CSV 1](https://www.kaggle.com/datasets/mahmoudshogaa/us-crime-rates-1960-2014/)
> * [CSV 2](https://www.kaggle.com/datasets/justin2028/unemployment-in-america-per-us-state)

#### Contenuto

> * Il CSV 1 contiene il numero di totale di crimini, divisi per categoria, avvenuti negli U.S.A dal 1960 al 2014.
>
> * Il CSV 2 contiene il numero e la percentuale dei cittadini con e senza lavoro, a partire dal 1970 fino al 2022.

---

## Scopo

L'obiettivo finale che ci siamo posti è quello di capire se c'è una correlazione fra i dati riportati dai due csv.
In particolare vogliamo dimostrare o confutare che:

> All'aumentare del tasso di disoccupazione aumentino anche i crimini, concentrandoci nei furti di vario tipo.

&nbsp;

Successivamente tramite i dati che abbiamo proveremo a fare una previsione se i crimini e la disoccupazione aumenteranno o diminuiranno nei prossimi anni.

---

# !!! Da cancellare prima della consegna !!!

## Idee implementative

* Dentro le slides di Statistics, c'è una parte intitolata: 'Measures of relationship between variables' (linear correlation coefficient = indice di correlazione di Person) e per verificare se i dati ottenuti si siano varificati per puro caso c'è: 'Hypothesis testing'.

* Anche la rigressione lineare è utile

* Sicuramente per il secondo obiettivo bisogna vedere le slides 'Time Series'

* Alcuni esempi per la correlazione fra due df sono negli esercizi di Statistica, nella cartella houses

In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import int64 #Ci servirà nella modifica del csv

#### Carichiamo i due CSV e li mostriamo

In [177]:
df_crime = pd.read_csv("CSV1.csv")
df_employment = pd.read_csv("CSV2.csv")

In [178]:
df_crime.head()

,Year,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault,Burglary,Larceny_Theft,Vehicle_Theft
0,1960,179323175,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,182992000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,185771000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,188483000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,191141000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


In [179]:
df_employment

,FIPS Code,State/Area,Year,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area
0,1,Alabama,1976,1,"2,605,000","1,484,555",57.0,"1,386,023",53.2,"98,532",6.6
1,2,Alaska,1976,1,"232,000","160,183",69.0,"148,820",64.1,"11,363",7.1
2,4,Arizona,1976,1,"1,621,000","964,120",59.5,"865,871",53.4,"98,249",10.2
3,5,Arkansas,1976,1,"1,536,000","889,044",57.9,"824,395",53.7,"64,649",7.3
4,6,California,1976,1,"15,621,000","9,774,280",62.6,"8,875,685",56.8,"898,595",9.2
...,...,...,...,...,...,...,...,...,...,...,...
29887,51,Virginia,2022,12,"6,862,585","4,470,272",65.1,"4,330,531",63.1,"139,741",3.1
29888,53,Washington,2022,12,"6,254,253","4,015,286",64.2,"3,832,769",61.3,"182,517",4.5
29889,54,West Virginia,2022,12,"1,434,789","784,323",54.7,"752,464",52.4,"31,859",4.1
29890,55,Wisconsin,2022,12,"4,753,700","3,068,610",64.6,"2,976,670",62.6,"91,940",3.0


#### Modifica CSV 2

Nel secondo CSV notiamo che ci sono molti dati, visto che il primo CSV raggruppa tutti i crimini degli interi U.S.A, andremo a raggruppare i dati del secondo CSV con la stessa logica. Inoltre 'dropperemo' le colonne che non ci serviranno.

Alla fine di tutti i passaggi avremo i seguenti campi:
* L'anno;
* Il numero totale di occupati;
* Il numero totale di disoccupati;
* Relative percentuali

In [180]:
#Iniziamo rimuovendo le colonne che non ci servono fin da subito
df_employment = df_employment.drop(columns=['FIPS Code', 'State/Area', 'Month', 'Percent (%) of State/Area\'s Population', 'Percent (%) of Labor Force Employed in State/Area', 'Percent (%) of Labor Force Unemployed in State/Area', 'Total Civilian Non-Institutional Population in State/Area', 'Total Civilian Labor Force in State/Area'])


#Ora dobbiamo modificare il tipo delle colonne, da String a Integer
df_employment[['Total Employment in State/Area', 'Total Unemployment in State/Area']] = df_employment[['Total Employment in State/Area', 'Total Unemployment in State/Area']].apply(lambda x: x.str.replace(',', '').str.strip().astype(int64)) #Prima di effettuare la conversione rimuoviamo le virgole ed eventuali spazi bianchi


#Ora raggruppiamo per anno e sommiamo i valori di ogni riga
df_employment = df_employment.groupby('Year').sum()


#Mostriamo il risultato
df_employment

,Total Employment in State/Area,Total Unemployment in State/Area
Year,,
1976,1135462506,96002740
1977,1176475401,90321855
1978,1227040923,80085071
1979,1262107980,78736988
1980,1267707156,97779196
1981,1281247062,105690680
1982,1270774705,136166005
1983,1285889696,136879177
1984,1338004869,109542934
